In [124]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [125]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'euw1'  # euw1 na1 kr oc1
LEAGUE='challengers'
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344

In [126]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [127]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,OcVNgTWf7sO6zRy6-QL2_16cRb3xqkjaVgKelQEYftY6mh...,b7z8JF9hKX-AbY-w_tzuBXO_0bqUdiTqh2gI4DqUveNf-9...,D-joNrgVY5D1HQjhdzC4ToHDMadctI8V772jODtt5LUCrr...,SLY L3SCoco,907,1656427876396,2,OcVNgTWf7sO6zRy6-QL2_16cRb3xqkjaVgKelQEYftY6mh...,SLY L3SCoco,694,I,61,32,False,False,True,False
1,nCZoLVHLgRd5Uxrjy6KG5LUOm4-CGeecs6NwY5BsImAdOaw,omnfI3WE5g0dlD23hTVe-DSIENKojAN-alG8WfJURC6wkA,6V0CDUZfVo2vAkHU1gTyn3mkpBtlafr194J1lPzWOOqQWD...,Sologesang,1640,1656452432000,113,nCZoLVHLgRd5Uxrjy6KG5LUOm4-CGeecs6NwY5BsImAdOaw,Sologesang,1399,I,179,104,True,False,False,True
2,HoVV4k9rNKapN22slGfTOZmjKsA-9fkZus-D4yx2QIoKMfw,m0se6fucovRqtVdGCCyw7WFDyIXYmRjm6vUJ1j5O0tFVHbM,6ybd-VomjU8w7_MZPYhOj3t-lHA12Czp7YKobZ7RIVKqJ7...,humishvperugnota,981,1656449969000,186,HoVV4k9rNKapN22slGfTOZmjKsA-9fkZus-D4yx2QIoKMfw,humishvperugnota,747,I,122,89,False,False,True,True
3,K_2so7mS7bVqtqScpOsbYOo-QAIKjkb9DPc8hu3-Y5VIs6E,_d_K1Cd2P4JNHL7tgukpaZWydKwdZbcKCDOptLCDT0zbUw,J7OtrN4teXk2_Z8UQUyh5H94Xcpl-LPJ5qnH8PT3THQBbp...,Matelas de Volta,539,1656468069000,394,K_2so7mS7bVqtqScpOsbYOo-QAIKjkb9DPc8hu3-Y5VIs6E,Matelas de Volta,1368,I,218,159,False,False,True,False
4,LHbAcZVWhNDvhAAoJqeDPHNMxaNLgq6HSL0j-M0dVTv10to,mSZeh687EUJwGnNvb8N5tmhsMNBQmAYZ8GARAt3u8QEfjJE,NtHt1bWrye9VM4T1v6WGiqW6g-gVYiIBCwWOpFVIDoEOl0...,Marcosvc22,1628,1656455561000,441,LHbAcZVWhNDvhAAoJqeDPHNMxaNLgq6HSL0j-M0dVTv10to,Marcosvc22,524,I,145,114,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Uls4NsKzgf4N82KZZPUwH3ti8eDn2Quei5EheJZRTATjzyI,LYjkl1CTq94gR9YTAum_WWKh7b7BvQF7Fq7iBcWdPCW5Z4k,MbfmP1QF7EpTn-pFERyENuhq0Mlv5WwJYCQLk5D7hKsIJQ...,Joshy Chan,917,1656428608000,171,Uls4NsKzgf4N82KZZPUwH3ti8eDn2Quei5EheJZRTATjzyI,Joshy Chan,506,I,117,88,False,False,True,True
118,hOFk1FwPtZ0TLFqeeg4V1kgWir7f-wwB6a01GQQyWlQIC3M,OyhfMsTDwuIV9p-BPQm2wRbfzJQWKL7qnGiL9sNS_Okuy7Y,tZ7G5_IM1YUAJ3EWeT9drtePbPgz_A-StV-x0c1nbUF4KC...,PickStar,4745,1656243826000,323,hOFk1FwPtZ0TLFqeeg4V1kgWir7f-wwB6a01GQQyWlQIC3M,PickStar,507,I,113,79,False,False,True,False
119,liQVbcVVwbtw38xwGYJBlJM-A8fGRQZ2Y--pTygXKkcc7NJh,Sc6tYrryRSfePmMONx7H1eDVjnbcecifD_Hr8hzxzufuIy4,djCH15wmd_P3MON867nLgdijP2I0bQ-A9stV28OQmJ_EHR...,Sinscere,3879,1656375493000,55,liQVbcVVwbtw38xwGYJBlJM-A8fGRQZ2Y--pTygXKkcc7NJh,Sinscere,756,I,98,45,False,False,True,True
120,EEeFY5ibd-24H9i6WoDIVPKjLTAwcnKI78p_5OM6wzNIZx4,bXF3FmjnfsjTxsIBG4FEC_z-XRczVHbHenRIqvQmft-PRj4,-ZNUkOtpvc---B9gTLmWgnCUbsm6kSSNnv6qy3_GvZPaWA...,5454XD,3836,1656343750000,353,EEeFY5ibd-24H9i6WoDIVPKjLTAwcnKI78p_5OM6wzNIZx4,5454XD,694,I,104,62,False,False,True,False


In [128]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [129]:
len(uniq_matches_id)

1957

In [130]:
len(matches_asset)

2857

In [131]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [132]:
len(uniq_matches)

1957

In [133]:
matches_asset[300]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [134]:
latest_matches = [match for match in uniq_matches if LATEST_RELEASE in match['info']['game_version']]

In [135]:
len(latest_matches)

1407

In [136]:
# latest_matches[0]['info']

In [137]:
matches_array = []

for match_row in latest_matches:
    match_id = match_row['metadata']['match_id']

    for participant in match_row['info']['participants']:
        match = {}
        match['match_id'] = match_id
        # match['level'] = participant['level']
        match['placement'] = participant['placement']
        # match['players_eliminated'] = participant['players_eliminated']
        # match['total_damage_to_players'] = participant['total_damage_to_players']

        for augment_index, augment in enumerate(participant['augments']):
            if augment == 'TFT7_Augment_GuildLootHR':
                augment = 'TFT7_Augment_BandOfThieves1'
            match[f'augment{augment_index}'] = augment

        for trait_index, trait in enumerate(participant['traits']):
            match[f'{trait["name"]}'] = trait["tier_current"]

        for unit_index, unit in enumerate(participant['units']):
            match[f'{unit["character_id"]}'] = unit["tier"]
            for item_index, item in enumerate(unit['itemNames']):
                match[f'{unit["character_id"]}_item{item_index}'] = item

        matches_array.append(match)

In [138]:
matches_array[0]
# len(matches_array)

{'match_id': 'EUW1_5944909932',
 'placement': 2,
 'augment0': 'TFT6_Augment_TrueTwos',
 'augment1': 'TFT6_Augment_CyberneticImplants2',
 'augment2': 'TFT6_Augment_SecondWind2',
 'Set7_Assassin': 1,
 'Set7_Bruiser': 1,
 'Set7_Cavalier': 2,
 'Set7_Dragonmancer': 0,
 'Set7_Guild': 3,
 'Set7_Legend': 0,
 'Set7_Mirage': 0,
 'Set7_Ragewing': 1,
 'Set7_Swiftshot': 1,
 'Set7_Tempest': 1,
 'Set7_Warrior': 1,
 'TFT7_Sejuani': 2,
 'TFT7_Qiyana': 2,
 'TFT7_Shen': 2,
 'TFT7_Shen_item0': 'TFT_Item_ChainVest',
 'TFT7_Shen_item1': 'TFT_Item_Shroud',
 'TFT7_Twitch': 2,
 'TFT7_Twitch_item0': 'TFT_Item_MadredsBloodrazor',
 'TFT7_Hecarim': 2,
 'TFT7_Talon': 2,
 'TFT7_Talon_item0': 'TFT_Item_RedBuff',
 'TFT7_Talon_item1': 'TFT_Item_InfinityEdge',
 'TFT7_Talon_item2': 'TFT7_Item_CavalierEmblemItem',
 'TFT7_Xayah': 2,
 'TFT7_Xayah_item0': 'TFT_Item_GuardianAngel',
 'TFT7_Xayah_item1': 'TFT_Item_InfinityEdge',
 'TFT7_Xayah_item2': 'TFT_Item_GuinsoosRageblade',
 'TFT7_Ornn': 2,
 'TFT7_Ornn_item0': 'TFT_Item_Gi

In [139]:
matches_league_df = pd.json_normalize(matches_array)

In [140]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bruiser,Set7_Cavalier,Set7_Dragonmancer,Set7_Guild,...,TFT7_Skarner_item0,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_Skarner_item1,TFT7_Skarner_item2,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Heimerdinger_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2
0,EUW1_5944909932,2,TFT6_Augment_TrueTwos,TFT6_Augment_CyberneticImplants2,TFT6_Augment_SecondWind2,1.0,1.0,2.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EUW1_5944909932,1,TFT6_Augment_PortableForge,TFT6_Augment_SalvageBin,TFT6_Augment_TargetDummies,NaN,0.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EUW1_5944909932,3,TFT7_Augment_AssassinEmblem,TFT6_Augment_TradeSectorPlus,TFT7_Augment_ScalescornEmblem,3.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EUW1_5944909932,4,TFT7_Augment_SwiftshotPressTheAttack,TFT6_Augment_RichGetRicherPlus,TFT7_Augment_BestFriends2,1.0,1.0,1.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EUW1_5944909932,7,TFT7_Augment_PandorasBench,TFT7_Augment_MikaelsGift,TFT6_Augment_SalvageBin,2.0,1.0,NaN,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11251,EUW1_5941855898,6,TFT7_Augment_BestFriends2,TFT7_Augment_MirageEmblem,TFT6_Augment_ComponentGrabBag,NaN,NaN,3.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11252,EUW1_5941855898,7,TFT6_Augment_HyperRoll,TFT6_Augment_ComponentGrabBag,TFT7_Augment_GuardianHeroicPresence,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11253,EUW1_5941855898,1,TFT7_Augment_UrfsGrabBag1,TFT6_Augment_TriForce2,TFT7_Augment_ScalescornEmblem,2.0,1.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11254,EUW1_5941855898,8,TFT6_Augment_ThrillOfTheHunt2,TFT6_Augment_TradeSectorPlus,TFT6_Augment_Diversify2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bruiser,Set7_Cavalier,Set7_Dragonmancer,Set7_Guild,...,TFT7_Skarner_item0,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_Skarner_item1,TFT7_Skarner_item2,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Heimerdinger_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2
0,EUW1_5944909932,2,TFT6_Augment_TrueTwos,TFT6_Augment_CyberneticImplants2,TFT6_Augment_SecondWind2,1.0,1.0,2.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EUW1_5944909932,1,TFT6_Augment_PortableForge,TFT6_Augment_SalvageBin,TFT6_Augment_TargetDummies,NaN,0.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EUW1_5944909932,3,TFT7_Augment_AssassinEmblem,TFT6_Augment_TradeSectorPlus,TFT7_Augment_ScalescornEmblem,3.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EUW1_5944909932,4,TFT7_Augment_SwiftshotPressTheAttack,TFT6_Augment_RichGetRicherPlus,TFT7_Augment_BestFriends2,1.0,1.0,1.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EUW1_5944909932,7,TFT7_Augment_PandorasBench,TFT7_Augment_MikaelsGift,TFT6_Augment_SalvageBin,2.0,1.0,NaN,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11251,EUW1_5941855898,6,TFT7_Augment_BestFriends2,TFT7_Augment_MirageEmblem,TFT6_Augment_ComponentGrabBag,NaN,NaN,3.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11252,EUW1_5941855898,7,TFT6_Augment_HyperRoll,TFT6_Augment_ComponentGrabBag,TFT7_Augment_GuardianHeroicPresence,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11253,EUW1_5941855898,1,TFT7_Augment_UrfsGrabBag1,TFT6_Augment_TriForce2,TFT7_Augment_ScalescornEmblem,2.0,1.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11254,EUW1_5941855898,8,TFT6_Augment_ThrillOfTheHunt2,TFT6_Augment_TradeSectorPlus,TFT6_Augment_Diversify2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))